In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_parquet("../data/processed/ais_filtered.parquet")
df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselType,Status,Length,Width,Draft
5,367606520,2024-12-21T00:00:00,29.11332,-90.19610,0.0,304.0,511.0,70.0,0.0,49.0,12.0,3.0
6,636093045,2024-12-21T00:00:00,29.34722,-89.48623,11.5,295.8,295.0,80.0,0.0,184.0,32.0,12.0
12,303923000,2024-12-21T00:00:00,30.02172,-94.00101,0.0,137.8,14.0,70.0,5.0,193.0,28.0,6.5
26,367605680,2024-12-21T00:00:00,29.08294,-91.89141,0.1,279.3,140.0,70.0,0.0,39.0,8.0,2.6
46,367342960,2024-12-21T00:00:01,30.17580,-93.31875,0.0,184.6,191.0,80.0,0.0,183.0,32.0,10.0


In [4]:
df["MMSI"].nunique()

983

In [3]:
df.describe()
#there are vessels with dimensions = 0, doesn't make sense. will be treated as NA

,MMSI,LAT,LON,SOG,COG,Heading,VesselType,Status,Length,Width,Draft
count,1.760960e+06,1.760960e+06,1.760960e+06,1.760960e+06,1.760960e+06,1.760960e+06,1.760960e+06,1.756416e+06,1.759060e+06,1.739643e+06,1.756416e+06
mean,4.280590e+08,2.921982e+01,-9.231607e+01,3.623438e+00,1.942072e+02,2.044532e+02,7.564911e+01,1.927006e+00,1.666169e+02,2.787740e+01,7.390024e+00
std,1.301785e+08,7.168659e-01,3.042460e+00,7.944395e+00,1.049578e+02,1.418212e+02,5.805783e+00,2.361887e+00,7.653744e+01,1.116135e+01,3.197983e+00
min,2.056850e+08,2.691230e+01,-9.750785e+01,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.520036e+08,2.894160e+01,-9.469550e+01,0.000000e+00,1.152000e+02,9.000000e+01,7.000000e+01,0.000000e+00,1.000000e+02,2.000000e+01,5.300000e+00
50%,3.682714e+08,2.929603e+01,-9.327950e+01,0.000000e+00,1.919000e+02,1.700000e+02,7.900000e+01,1.000000e+00,1.830000e+02,3.100000e+01,7.700000e+00
75%,5.380095e+08,2.974485e+01,-9.020140e+01,7.600000e+00,2.868000e+02,3.040000e+02,8.000000e+01,5.000000e+00,2.130000e+02,3.200000e+01,9.300000e+00
max,6.421220e+08,3.061882e+01,-8.239115e+01,1.023000e+02,3.600000e+02,5.110000e+02,8.900000e+01,1.500000e+01,3.360000e+02,9.400000e+01,2.550000e+01


# Duplicates

In [3]:
print(f"There are {df.duplicated().sum()} doublons. Manually confirmed for some of them")
df = df.drop_duplicates()

df = df.drop(columns="VesselType") #shouldn't be useful now.


There are 186 doublons. Manually confirmed for some of them


# Missing values

In [4]:
# % missing values per feature
df.isna().sum()/len(df) * 100
# not a lot, worst is 1.21% in Width. Length Width and Draft arent the core features either. Status is a bit more problematic


MMSI            0.000000
BaseDateTime    0.000000
LAT             0.000000
LON             0.000000
SOG             0.000000
COG             0.000000
Heading         0.000000
Status          0.258068
Length          0.107907
Width           1.210661
Draft           0.258068
dtype: float64

In [5]:
# check if missing values are concentrated in some boats.

#calculating number of na per feature and per vessel
na_per_vessel_feature = df[["Status", "Length", "Width", "Draft", "MMSI"]].groupby("MMSI").agg(lambda x: x.isna().sum())
# number of lines (ping) per vessel
ping_per_vessel = df.groupby("MMSI").count()["BaseDateTime"]

# % of pings with na, per feature et per vessel
na_pc = na_per_vessel_feature.divide(ping_per_vessel, axis= 0)*100
#filtering on vessels with na
na_pc.loc[na_pc.any(axis= 1) > 0,:]


,Status,Length,Width,Draft
MMSI,,,,
205700000,0.0,0.0,100.0,0.0
249579000,0.0,0.0,100.0,0.0
319093800,0.0,0.0,100.0,0.0
368001940,100.0,0.0,0.0,100.0
368203510,100.0,0.0,100.0,100.0
368287410,100.0,0.0,0.0,100.0
368305620,100.0,0.0,0.0,100.0
477669500,0.0,0.0,100.0,0.0
538005805,0.0,0.0,100.0,0.0


- There are 15 boats with na
- When values are missing they are missing for the entire sequence. It's not a issue of pings, but an issue of boat
- The status feature is crucial for route prediction, so we'll drop boats with that missing info (not a lot of boat)
- dimension features should be less crucials and fixed for one boat. will impoute using median. 

In [6]:
na_values = {"Length": df["Length"].median(),
             "Width": df["Width"].median(),
             "Draft": df["Draft"].median()}

df = df.fillna(value= na_values)


In [7]:
df = df.dropna(subset=["Status"])

### Resampling

The idea is to make sure the time intervals are all the same

In [ ]:
#convert to date time
df["BaseDateTime"] = pd.to_datetime(df["BaseDateTime"])
# sort by time and vessesl
df = df.sort_values(by=["MMSI", "BaseDateTime"], ascending= True)

# resampling every 5 min per vessels
df = df.set_index("BaseDateTime")

#resample in 5min bin per boats
df_resampled = df.groupby("MMSI").resample("5min").last()


# #interpolate if this creates NA
df_resampled = df_resampled.interpolate("linear")
#get MMSI and BaseDateTime back to features
df_resampled = df_resampled.drop(columns= "MMSI")
df = df_resampled.reset_index()
df

/var/folders/dk/v6_xfc950bb08vyf1lm9v0x00000gn/T/ipykernel_85521/1303395929.py:10: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resampled = df.groupby("MMSI").resample("5min").last()


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,Status,Length,Width,Draft
0,205685000,2024-12-23 13:55:00,28.584470,-94.406530,15.30,6.40,11.0,0.0,180.0,30.0,9.4
1,205685000,2024-12-23 14:00:00,28.611775,-94.402755,15.15,6.75,10.0,0.0,180.0,30.0,9.4
2,205685000,2024-12-23 14:05:00,28.639080,-94.398980,15.00,7.10,9.0,0.0,180.0,30.0,9.4
3,205685000,2024-12-23 14:10:00,28.654760,-94.396920,13.50,7.10,9.0,0.0,180.0,30.0,9.4
4,205685000,2024-12-23 14:15:00,28.670440,-94.394860,12.00,7.10,9.0,0.0,180.0,30.0,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1110538,642122014,2024-12-26 23:35:00,27.575770,-96.376730,13.00,265.10,261.0,0.0,183.0,32.0,9.0
1110539,642122014,2024-12-26 23:40:00,27.574140,-96.398390,13.00,265.50,261.0,0.0,183.0,32.0,9.0
1110540,642122014,2024-12-26 23:45:00,27.572780,-96.415980,12.90,265.30,260.0,0.0,183.0,32.0,9.0
1110541,642122014,2024-12-26 23:50:00,27.571310,-96.434800,12.90,264.30,260.0,0.0,183.0,32.0,9.0


### Necessary functions for further exploration
- lag features
- scoring

In [ ]:
def create_time_series_features(df, prediction_horizon_min=30):
    """
    Create lagged features and target proportional to prediction horizon.

    Parameters:
    -----------
    df : DataFrame
        Must be resampled at 5-min intervals, sorted by MMSI and BaseDateTime
    prediction_horizon_min : int
        Prediction horizon in minutes (30, 60, 720, etc.)

    Returns:
    --------
    DataFrame with lagged features and targets
    """
    df_features = df.copy()

    # Calculate lag intervals (in number of 5-min steps)
    # Proportional: 1/6, 1/2, and full horizon
    horizon_steps = prediction_horizon_min // 5  # Convert minutes to 5-min steps

    lag_1 = max(1, horizon_steps // 6)    # ~1/6 of horizon
    lag_2 = max(2, horizon_steps // 2)    # ~1/2 of horizon
    lag_3 = horizon_steps                 # Full horizon (for reference)

    print(f"Prediction horizon: {prediction_horizon_min} min ({horizon_steps} steps)")
    print(f"Lag intervals: {lag_1*5}min, {lag_2*5}min, {lag_3*5}min")

    # Create lagged features
    for var in ['LAT', 'LON', 'SOG', 'COG']:
        df_features[f'{var}_lag_{lag_1*5}min'] = df_features.groupby('MMSI')[var].shift(lag_1)
        df_features[f'{var}_lag_{lag_2*5}min'] = df_features.groupby('MMSI')[var].shift(lag_2)
        df_features[f'{var}_lag_{lag_3*5}min'] = df_features.groupby('MMSI')[var].shift(lag_3)

    # Create targets
    df_features['target_LAT'] = df_features.groupby('MMSI')['LAT'].shift(-horizon_steps)
    df_features['target_LON'] = df_features.groupby('MMSI')['LON'].shift(-horizon_steps)

    # Drop NaN
    df_features = df_features.dropna().reset_index(drop=True)

    return df_features


Prediction horizon: 30 min (6 steps)
Lag intervals: 5min, 15min, 30min
Prediction horizon: 60 min (12 steps)
Lag intervals: 10min, 30min, 60min
Prediction horizon: 720 min (144 steps)
Lag intervals: 120min, 360min, 720min


# Model Training & Evaluation Pipeline

**Approach:** Predict LAT and LON directly using MultiOutputRegressor 

**Pipeline steps:**
1. Train/test split (temporal or by vessel)
2. Setup MultiOutputRegressor(XGBRegressor()) - trains 2 models (one for LAT, one for LON)
3. Train on features → predict [LAT_target_30min, LON_target_30min]
4. Evaluate using Haversine distance MAE (km, not degrees)

**Baseline comparison:**
- Calculate naive baseline: extrapolate position using current SOG and COG
- Compare XGBoost MAE vs baseline MAE

**Optional enhancements (if time permits):**
- GridSearchCV with Haversine MAE as custom scorer
- Add rolling features (mean/std of SOG, COG over 30min)
- Add distance traveled feature (requires GeoDataFrame)
- Feature importance analysis


## Definition of necessary functions for metrics and estimations

In [14]:
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, GridSearchCV, cross_val_score, cross_validate
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor


# ============================================================================
# METRIC FUNCTIONS
# ============================================================================

def haversine_distance(LAT_true, LON_true, LAT_pred, LON_pred):
    """
    Calculate great-circle distance between two points on Earth using Haversine formula.

    Parameters:
    -----------
    LAT_true, LON_true : array-like
        True latitude and longitude coordinates in degrees
    LAT_pred, LON_pred : array-like
        Predicted latitude and longitude coordinates in degrees

    Returns:
    --------
    float
        Great-circle distance in kilometers
    """
    earth_radius = 6371  # Earth mean radius in kilometers

    # Convert degrees to radians
    LAT_true_rad = np.radians(LAT_true)
    LON_true_rad = np.radians(LON_true)
    LAT_pred_rad = np.radians(LAT_pred)
    LON_pred_rad = np.radians(LON_pred)

    # Calculate differences
    d_LAT = LAT_pred_rad - LAT_true_rad
    d_LON = LON_pred_rad - LON_true_rad

    # Haversine formula
    a = (np.sin(d_LAT / 2.0)**2 + np.cos(LAT_true_rad)*np.cos(LAT_pred_rad)*np.sin(d_LON/2.0)**2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))  # Angular distance in radians
    d = c * earth_radius  # Convert to kilometers
    return d


def haversine_mae(y_true, y_pred):
    """
    Calculate Mean Absolute Error using Haversine distance between positions.

    This metric evaluates prediction quality in km rather than degrees.

    Parameters:
    -----------
    y_true : array, shape (n_samples, 2)
        True positions where column 0 = LAT, column 1 = LON (in degrees)
    y_pred : array, shape (n_samples, 2)
        Predicted positions where column 0 = LAT, column 1 = LON (in degrees)

    Returns:
    --------
    float
        Mean Absolute Error in kilometers
    """

    #make sure we're converting in numpy
    y_true= np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    # Calculate haversine distance for each prediction
    mae = np.mean(abs(haversine_distance(y_true[:,0],
                                         y_true[:,1],
                                         y_pred[:,0],
                                         y_pred[:,1]) ))
    return mae


# Create sklearn-compatible scorer (negated for GridSearchCV minimization)
haversine_scorer = make_scorer(haversine_mae, greater_is_better= False)


# ============================================================================
# BASELINE MODEL (NAIVE PREDICTOR)
# ============================================================================

def position_extrapolation(df: pd.DataFrame):
    """
    Naive baseline: extrapolate position based on linear displacement over last 30 minutes.

    Assumes constant velocity: future_displacement = past_displacement
    Mathematically: position(t+30) = position(t) + [position(t) - position(t-30)]

    Parameters:
    -----------
    df : DataFrame
        Must contain columns: LAT, LON, LAT_lag_30, LON_lag_30

    Returns:
    --------
    LAT_pred, LON_pred : Series
        Predicted latitude and longitude 30 minutes ahead
    """
    # Calculate displacement over the last 30 minutes
    dLAT = df["LAT"] - df["LAT_lag_30"]
    dLON = df["LON"] - df["LON_lag_30"]

    # Extrapolate: assume same displacement for next 30 minutes
    LAT_pred = df["LAT"] + dLAT
    LON_pred = df["LON"] + dLON

    return LAT_pred, LON_pred


## Train / Test split keeping boats in same group

In [15]:
#define features df, targets (2 targets) and the group to guide split
X = df.drop(columns=[ "target_LAT", "target_LON"])
y = df[["target_LAT", "target_LON"]]
groups = df["MMSI"]
X.head(10)

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,Status,Length,Width,...,LAT_lag_15,LON_lag_15,LAT_lag_30,LON_lag_30,SOG_lag_5,COG_lag_5,SOG_lag_15,COG_lag_15,SOG_lag_30,COG_lag_30
0,205685000,2024-12-23 14:25:00,28.70223,-94.39070,4.9,22.2,37.0,0.0,180.0,30.0,...,28.65476,-94.39692,28.584470,-94.406530,10.5,7.1,13.5,7.1,15.30,6.40
1,205685000,2024-12-23 14:30:00,28.70778,-94.38800,3.1,26.3,47.0,0.0,180.0,30.0,...,28.67044,-94.39486,28.611775,-94.402755,4.9,22.2,12.0,7.1,15.15,6.75
2,205685000,2024-12-23 14:35:00,28.70888,-94.38740,2.8,24.7,48.0,0.0,180.0,30.0,...,28.68612,-94.39280,28.639080,-94.398980,3.1,26.3,10.5,7.1,15.00,7.10
3,205685000,2024-12-23 14:40:00,28.71272,-94.38537,2.7,36.4,63.0,0.0,180.0,30.0,...,28.70223,-94.39070,28.654760,-94.396920,2.8,24.7,4.9,22.2,13.50,7.10
4,205685000,2024-12-23 14:45:00,28.71730,-94.37955,6.2,66.0,62.0,0.0,180.0,30.0,...,28.70778,-94.38800,28.670440,-94.394860,2.7,36.4,3.1,26.3,12.00,7.10
5,205685000,2024-12-23 14:50:00,28.72068,-94.37627,8.3,17.8,20.0,0.0,180.0,30.0,...,28.70888,-94.38740,28.686120,-94.392800,6.2,66.0,2.8,24.7,10.50,7.10
6,205685000,2024-12-23 14:55:00,28.74377,-94.37807,13.0,347.5,350.0,0.0,180.0,30.0,...,28.71272,-94.38537,28.702230,-94.390700,8.3,17.8,2.7,36.4,4.90,22.20
7,205685000,2024-12-23 15:00:00,28.76378,-94.38320,13.7,348.3,350.0,0.0,180.0,30.0,...,28.71730,-94.37955,28.707780,-94.388000,13.0,347.5,6.2,66.0,3.10,26.30
8,205685000,2024-12-23 15:05:00,28.78305,-94.38802,14.0,346.1,350.0,0.0,180.0,30.0,...,28.72068,-94.37627,28.708880,-94.387400,13.7,348.3,8.3,17.8,2.80,24.70
9,205685000,2024-12-23 15:10:00,28.80278,-94.39300,14.0,347.4,349.0,0.0,180.0,30.0,...,28.74377,-94.37807,28.712720,-94.385370,14.0,346.1,13.0,347.5,2.70,36.40


In [16]:



#define splitter making sure that each boat is part of only train or test
gss = GroupShuffleSplit(n_splits=1, test_size= 0.2)

# obtain the indexse from the generator returned by .split()
for train_idx, test_idx in gss.split(X,y,groups):
    X_train, X_test = X.iloc[train_idx,:], X.iloc[test_idx,:]
    y_train, y_test = y.iloc[train_idx,:], y.iloc[test_idx,:]


## Baseline Score

In [18]:

# Generate baseline predictions using linear extrapolation
LAT_pred, LON_pred = position_extrapolation(X_test)

# Prepare arrays for metric calculation
y_true = y_test.values  # shape (n, 2): [LAT, LON]
y_pred_baseline = np.column_stack([LAT_pred, LON_pred])  # shape (n, 2)

# Calculate Haversine MAE
baseline_mae = haversine_mae(y_true, y_pred_baseline)

# Display results

print("BASELINE MODEL PERFORMANCE")
print(f"Model: Linear extrapolation")
print(f"Prediction horizon: 30 minutes")
print(f"Test set size: {len(y_test):,} samples \n {X_test["MMSI"].nunique()} boats")
print(f"\nMean Absolute Error: {baseline_mae:.3f} km")
print(f"                    = {baseline_mae*1000:.1f} meters")


BASELINE MODEL PERFORMANCE
Model: Linear extrapolation
Prediction horizon: 30 minutes
Test set size: 217,682 samples 
 195 boats

Mean Absolute Error: 0.929 km
                    = 928.5 meters


## XGBOOST

In [19]:
groups_crossval = X_train["MMSI"]
X_train = X_train.drop(columns=["MMSI", "BaseDateTime"])

 forest

In [20]:
#For this approach we need to drop the boat ID as feature


model = MultiOutputRegressor(XGBRegressor(n_estimators=200,
        max_depth=6,
        learning_rate=0.1,     # Learning rate standard
        subsample=0.8,         # Bagging
        colsample_bytree=0.8,  # Feature sampling
        random_state=42))
gkf = GroupKFold(n_splits= 5)

scores = cross_val_score(model,
                         X_train,
                         y_train,
                         cv= gkf,
                         groups= groups_crossval,
                         scoring=haversine_scorer,
                         n_jobs= -1)

print(f"crossval scores: {-scores}")
print(f"average haversine MAE: {-scores.mean():.2f}")


crossval scores: [2.49911599 2.3946223  2.59274721 2.17475136 3.04657688]
average haversine MAE: 2.54


## LINREG

In [22]:
from sklearn.linear_model import LinearRegression

# LinearRegression avec cross-validation
model_lr = MultiOutputRegressor(LinearRegression())

# S'assurer que les indices correspondent
scores_lr = cross_val_score(
    model_lr,
    X_train,
    y_train,
    cv=gkf,
    groups=groups_crossval,
    scoring=haversine_scorer,
    n_jobs=-1
)

print(f"LinearRegression cross-validation scores: {-scores_lr}")
print(f"LinearRegression MAE: {-scores_lr.mean():.2f} km ± {scores_lr.std():.2f}")
print(f"Baseline MAE: 0.75 km")
print(f"Ratio: {-scores_lr.mean() / 0.75:.2f}x")

/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: Runtime

LinearRegression cross-validation scores: [2.06386398 2.09011286 1.79658993 1.88600113 1.9056542 ]
LinearRegression MAE: 1.95 km ± 0.11
Baseline MAE: 0.75 km
Ratio: 2.60x


/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/pablorougerie/.pyenv/versions/3.12.9/envs/vessel_tracker/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: Runtime

## Conclusions

**Data quality:**
- 186 duplicates removed
- Missing values concentrated in 15 vessels (entire sequences)
- Status NA → vessels dropped (9 boats)
- Dimensions NA → median imputation

**Data preparation:**
- Resampling to 5-min intervals with linear interpolation
- Lag features created proportionally to prediction horizon
- Train/test split by vessel (GroupShuffleSplit) to ensure generalization

**Initial results (30-min horizon):**
- Baseline (linear extrapolation): **0.929 km MAE**
- XGBoost: **2.54 km MAE** (cross-validation)
- Linear Regression: **1.95 km MAE** (cross-validation)

At short horizons, linear extrapolation significantly outperforms ML models. This suggests vessel trajectories are highly linear over 30 minutes. Further analysis will test longer prediction horizons to identify when ML becomes beneficial.

*Will explore different horizon of predictions in Notebook 2*

